In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def _make_mask(l):
    m = np.ones((l, l))
    m[np.tril_indices(l)] = 0
    return m

In [ ]:
df = pd.read_pickle('result/debug/pred_ep_49.pkl.gz')

In [ ]:
df.head()

In [ ]:
# row = df.iloc[-10]
row = df.iloc[8]

In [ ]:
# target = row['target']
# pred = row['pred']

# # apply 'hard-mask'  (lower triangle)
# assert target.shape == pred.shape   # channel x h x w
# assert target.shape[1] == target.shape[2]
# m = _make_mask(target.shape[1])

# # hacky way to infer padding mask location
# # by finding out how many consecutive columns are all 0's in target channel 0
# tmp = np.sum(target[0, :, :], axis=0)
# first_nonzero_idx_from_right = next(i for i in range(len(tmp)) if tmp[-(i+1)] > 0)
# m[:, -first_nonzero_idx_from_right:] = 0
# m[-first_nonzero_idx_from_right:, :] = 0

# # repeat mask on channel
# m = m[np.newaxis, :, :]
# m = np.repeat(m, target.shape[0], axis=0)

# # apply mask
# target = target * m
# pred = pred * m

# # TODO save mask from training, crop out masked area, also apply 'hard mask' (lower triangle) on prediction  

In [ ]:
target = row['target']
pred = row['pred']

# hacky way to infer padding location
# by finding out how many consecutive columns are all 0's in target channel 0
tmp = np.sum(target[0, :, :], axis=0)
first_nonzero_idx_from_right = next(i for i in range(len(tmp)) if tmp[-(i+1)] > 0)

# crop
target = target[:, :-first_nonzero_idx_from_right, :-first_nonzero_idx_from_right]
pred = pred[:, :-first_nonzero_idx_from_right, :-first_nonzero_idx_from_right]

# apply 'hard-mask'  (lower triangle)
assert target.shape == pred.shape   # channel x h x w
assert target.shape[1] == target.shape[2]
m = _make_mask(target.shape[1])
# repeat mask on channel
m = m[np.newaxis, :, :]
m = np.repeat(m, target.shape[0], axis=0)

# apply mask
target = target * m
pred = pred * m

# TODO save mask from training, crop out masked area, also apply 'hard mask' (lower triangle) on prediction  

In [ ]:
print(target.shape)
print(pred.shape)

In [ ]:
# this is known from data processing
# not_local_structure, stem, internal_loop (include bulges), hairpin loop, is_corner
target_names = ['not_ss', 'stem', 'i_loop', 'h_loop', 'corner']
assert len(target_names) == target.shape[0]
assert len(target_names) == pred.shape[0]

fig = make_subplots(rows=len(target_names), cols=2 ,print_grid=False, shared_yaxes=True, shared_xaxes=True)


for i in range(len(target_names)):
    # target
    tmp_fig = px.imshow(target[i, :, :])
    fig.append_trace(tmp_fig.data[0], i+1, 1)
    fig['layout']['xaxis{}'.format(2*i+1)].update(title=target_names[i])
    fig['layout']['yaxis{}'.format(2*i+1)]['autorange'] = "reversed"
    # pred
    tmp_fig = px.imshow(pred[i, :, :])
    fig.append_trace(tmp_fig.data[0], i+1, 2)
    fig['layout']['xaxis{}'.format(2*i+2)].update(title=target_names[i])
    fig['layout']['yaxis{}'.format(2*i+1)]['autorange'] = "reversed"
    # TODO apply hard mask?


# fig['layout']['yaxis1'].update(title='y-axis')
fig['layout'].update(height=400*len(target_names), width=400*2, title='type: {}'.format(row.subset))

fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()